For some reason the pre-process code for 1-25000 and master.csv was lost. (since post-processed files are stored in Google Drive)  
The code would be similar to what I've done to 30000-50000 and train.csv. I would try to reproduce these code for assignment delivery purpose but I can't guarantee that they would run correctly since they took too long to run.  
(Have to admit this notebook looks messy)  
All code below "#Read y" block should work properly.The post-processed csvs ("train2" and "test2") will be listed below to show that indeed all 45000 pictures are correctly processed.  

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


In [ ]:
# Many imports are redundant imports
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import LeakyReLU, PReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D, AveragePooling2D, ZeroPadding2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from sklearn import metrics
from sklearn.utils import shuffle
from PIL import Image
from tqdm.notebook import trange, tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input # A shame we didn't try transfer learning
from tensorflow.keras.applications import InceptionResNetV2 # A shame we didn't try transfer learning
# NN-related packs end here
import os 
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import shutil
import random
import math
from google.colab.patches import cv2_imshow

WORKING_DIR="/content/drive/My Drive/"

TRAIN_DATA="/content/drive/My Drive/train/"
PROCESSED_TRAIN_DATA="/content/drive/My Drive/proctrain/"

TEST_DATA="/content/drive/My Drive/test/"
PROCESSED_TEST_DATA="/content/drive/My Drive/proctest/"

np.random.seed(42)
tf.random.set_seed(42)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find("failed")>=0:
  print("No GPU")
else:
  print(gpu_info)

# Following are depreciated folders:
# ORIGINAL_PATH="/content/drive/My Drive/Colab Notebooks/Kaggle/pre_release/clips/" #Folder of original image
# PROCESSED_PATH="/content/drive/My Drive/Colab Notebooks/Kaggle/pre_release/processed/" #Folder of processed image
# TRIAL_PROCESSED_PATH="/content/drive/My Drive/Colab Notebooks/Kaggle/pre_release/grey/" #Experimental grey images
# RELEASE_PATH="/content/drive/My Drive/Colab Notebooks/Kaggle/release/clips/"

In [ ]:
# Create path cache to avoid I/O error.
# Re-run multiple times until it stop giving Google Drive timeout error

os.listdir(TRAIN_DATA)
os.listdir(PROCESSED_TRAIN_DATA)
os.listdir(TEST_DATA)
os.listdir(PROCESSED_TEST_DATA)


# Following are depreciated folders:
# os.listdir('/content/drive/My Drive/Colab Notebooks/Kaggle/pre_release/')
# os.listdir(ORIGINAL_PATH)
# os.listdir(PROCESSED_PATH)
# os.listdir(TRIAL_PROCESSED_PATH)
# os.listdir(RELEASE_PATH)

In [0]:
# DEPRECIATED: Use clip.zip instead
# # Download file of essential images, only run one time
# # Press Ctrl + / to un-comment
# import requests  
# file_url = #REDACTED#
    
# r = requests.get(file_url, stream = True)  
 
# with open("/content/drive/My Drive/Colab Notebooks/Kaggle/release/clips-data-2020.zip", "wb") as file:  
#     for block in r.iter_content(chunk_size = 1024): 
#          if block:  
#              file.write(block)  

# #Unzip, only run one time
# !unzip -uq "/content/drive/My Drive/Colab Notebooks/Kaggle/release/clips-data-2020.zip" -d "/content/drive/My Drive/Colab Notebooks/Kaggle/release/"

# # Download file of optional images, only run one time
# # Press Ctrl + / to un-comment
# import requests  
# file_url = #REDACTED#
    
# r = requests.get(file_url, stream = True)  
 
# with open("/tmp/clips-pre-data-2020.zip", "wb") as file:  
#     for block in r.iter_content(chunk_size = 1024): 
#          if block:  
#              file.write(block)  

# #Unzip, only run one time
# !unzip -uq "/content/drive/My Drive/Colab Notebooks/Kaggle/pre_release/clips-pre-data-2020.zip" -d "/content/drive/My Drive/Colab Notebooks/Kaggle/pre_release/"

In [0]:
# DEPRECIATED: SEE NEXT BLOCK
# # Download file of essential images, only run one time
# # Press Ctrl + / to un-comment
# import requests  
# file_url = #REDACTED#
    
# r = requests.get(file_url, stream = True)  
 
# with open("/tmp/clips-data-2020.zip", "wb") as file:  
#     for block in r.iter_content(chunk_size = 1024): 
#          if block:  
#              file.write(block)

# # Alternative unzip, during the process, DO NOT OPEN GOOGLE DRIVE, keep this window open even after the unzip process and wait
# shutil.unpack_archive("/tmp/clips-data-2020.zip","/tmp")

# # Download file of optional images, only run one time
# # Press Ctrl + / to un-comment
# import requests  
# file_url = #REDACTED#
    
# r = requests.get(file_url, stream = True)  
 
# with open("/tmp/clips-pre-data-2020.zip", "wb") as file:  
#     for block in r.iter_content(chunk_size = 1024): 
#          if block:  
#              file.write(block) 
# # Alternative unzip, during the process, DO NOT OPEN GOOGLE DRIVE, keep this window open even after the unzip process and wait
# shutil.unpack_archive("/tmp/clips-pre-data-2020.zip","/tmp")

In [0]:
# Thanks to Google Drive's "insane" I/O, I have to manually downloaded both datasets, put clips separately into "train" and "test" folders, and re-zip and upload the zip file to the google drive.
shutil.unpack_archive("/content/drive/My Drive/clips.zip","/content/drive/My Drive/")

In [ ]:
# Check unzip integrity, should wait for a while to run this block after unzip
print("Train Data: 1-25000, 30001-50000")
for i in range(29990,50002):
  if not os.path.exists(os.path.join(TRAIN_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")
for i in range(1,25001):
  if not os.path.exists(os.path.join(TRAIN_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")

print("Test Data: 25001-30000")
for i in range(24990,30002):
  if not os.path.exists(os.path.join(TEST_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")

# Expected result: no 29990-30000,50001 in train data; no 24990-25000, 30001 in test data

In [0]:
# Image Subtract Preprocess
# Only need to run once
base=cv2.imread(os.path.join(TRAIN_DATA,"clips-30047.png"))

raise RuntimeError # Foolproof, if you want to run, comment this line

for i in tqdm(range(1,25001)):
  if not os.path.exists(os.path.join(TRAIN_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")
  else:
    test=cv2.imread(os.path.join(TRAIN_DATA,f"clips-{i}.png"))
    diff=cv2.subtract(base,test)
    diff=cv2.cvtColor(diff,cv2.COLOR_BGR2GRAY)

    output_filename = os.path.join((PROCESSED_TRAIN_DATA),f"clips-{i}.png")
    writestatus=cv2.imwrite(output_filename,diff)

for i in tqdm(range(30000,50002)):
  if not os.path.exists(os.path.join(TRAIN_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")
  else:
    test=cv2.imread(os.path.join(TRAIN_DATA,f"clips-{i}.png"))
    diff=cv2.subtract(base,test)
    diff=cv2.cvtColor(diff,cv2.COLOR_BGR2GRAY)

    output_filename = os.path.join((PROCESSED_TRAIN_DATA),f"clips-{i}.png")
    writestatus=cv2.imwrite(output_filename,diff)

In [0]:
# Test Image Subtract
# Only need to run once
base=cv2.imread(os.path.join(TRAIN_DATA,"clips-30047.png"))

raise RuntimeError # Foolproof, if you want to run, comment this line
for i in tqdm(range(25001,30002)):
  if not os.path.exists(os.path.join(TEST_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")
  else:
    test=cv2.imread(os.path.join(TEST_DATA,f"clips-{i}.png"))
    diff=cv2.subtract(base,test)
    diff=cv2.cvtColor(diff,cv2.COLOR_BGR2GRAY)

    output_filename = os.path.join((PROCESSED_TEST_DATA),f"clips-{i}.png")
    writestatus=cv2.imwrite(output_filename,diff)

In [0]:
# Data frame merging and preprocess
# Only need to run once
df=pd.read_csv(os.path.join(WORKING_DIR,'train.csv'))
# Change id format to match imageDataGenerator requirement
df["id"]=df['id'].apply(lambda x: f"clips-{x}.png")
df

In [ ]:
df_m=pd.read_csv(os.path.join(WORKING_DIR,'master.csv'))
# Change id format to match imageDataGenerator requirement
df_m["id"]=df_m['id'].apply(lambda x: f"clips-{x}.png")
df_m

In [0]:
# Calculate average pixel
# Only need to run once

raise RuntimeError # Foolproof, if you want to run, comment this line

avg_pixel=[]
for i in tqdm(range(1,25001)):
  if not os.path.exists(os.path.join(PROCESSED_TRAIN_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")
  else:
    test=cv2.imread(os.path.join(PROCESSED_TRAIN_DATA,f"clips-{i}.png"),cv2.IMREAD_GRAYSCALE)
    avg_pixel.append(np.mean(test))
df_avg_pixel=pd.DataFrame(avg_pixel)
df_m=pd.concat([df_m,df_avg_pixel],axis=1,ignore_index=True)
df_m=df_m.rename(columns={0:"id",1:"clip_count",2:"avg_pixel"})

avg_pixel=[]
for i in tqdm(range(30000,50002)):
  if not os.path.exists(os.path.join(PROCESSED_TRAIN_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")
  else:
    test=cv2.imread(os.path.join(PROCESSED_TRAIN_DATA,f"clips-{i}.png"),cv2.IMREAD_GRAYSCALE)
    avg_pixel.append(np.mean(test))
df_avg_pixel=pd.DataFrame(avg_pixel)
df=pd.concat([df,df_avg_pixel],axis=1,ignore_index=True)
df=df.rename(columns={0:"id",1:"clip_count",2:"avg_pixel"})

df=pd.concat([df_m,df],axis=0,ignore_index=True)
df=df.sample(frac=1).reset_index(drop=True)
df.to_csv('/content/drive/My Drive/train2.csv',index=False)

In [0]:
# Data frame preprocess
# Only need to run once
df_test=pd.read_csv(os.path.join(WORKING_DIR,'test.csv'))
# Change id format to match imageDataGenerator requirement
df_test["id"]=df_test['id'].apply(lambda x: f"clips-{x}.png")
df_test

In [0]:
# Calculate average pixel
# Only need to run once
avg_pixel_test=[]
for i in tqdm(range(25001,30001)):
  if not os.path.exists(os.path.join(PROCESSED_TEST_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")
  else:
    test=cv2.imread(os.path.join(PROCESSED_TEST_DATA,f"clips-{i}.png"),cv2.IMREAD_GRAYSCALE)
    avg_pixel_test.append(np.mean(test))
df_avg_pixel_test=pd.DataFrame(avg_pixel_test)
df_test=pd.concat([df_test,df_avg_pixel_test],axis=1,ignore_index=True)
df_test=df_test.rename(columns={0:"id",1:"avg_pixel"})
df_test.to_csv('/content/drive/My Drive/test2.csv',index=False)
df_avg_pixel_test.describe()

In [ ]:
# Read y
df=pd.read_csv(os.path.join(WORKING_DIR,'train2.csv'))
df_test=pd.read_csv(os.path.join(WORKING_DIR,'test2.csv'))

cutoff33=df_test.quantile(q=0.33)[0]
cutoff66=df_test.quantile(q=0.66)[0]

print(cutoff33,cutoff66)
# 7.846227722167969 14.933803405761719

# Use 30047 as base image
# df[df["clip_count"]]==0
# # All picture's background has exact same pixel.
# # We compare all picture pixel with one 0-clip picture, we force the pixels to be [255,255,255,255] if the pixles match
# image1=Image.open("/content/drive/My Drive/Colab Notebooks/Kaggle/pre_release/clips/clips-48.png")
# image_array1=np.asarray(image1)
# image2=Image.open("/content/drive/My Drive/Colab Notebooks/Kaggle/pre_release/clips/clips-24831.png")
# image_array2=np.asarray(image2)
# np.array_equal(image_array1,image_array2)
# True



In [ ]:
display(df)
# This is to prove that all training data is used and processed
display(df_test)

In [0]:
# DEPRECIATED: TOO LONG RUNTIME
# #trial picture clean
# base = np.asarray(Image.open(os.path.join(TRAIN_DATA,"clips-30047.png"))).copy()

# for filename in tqdm(df3["id"]):
#   test = np.asarray(Image.open(os.path.join(TRAIN_DATA,filename))).copy()
#   for i in range(256):
#     for j in range(256):
#       if (np.std([test[i,j][0],test[i,j][1],test[i,j][2]])) >= 13.5 and (test[i,j][2]>test[i,j][1]) and (test[i,j][2]>test[i,j][0]):
#         test[i,j] = np.array([255,255,255,255],dtype="uint8")
#       elif (test[i,j][0]>test[i,j][1]) and (test[i,j][0]>test[i,j][2]):
#         if (np.std([test[i,j][0],test[i,j][1],test[i,j][2]])) >= 14 and (test[i,j][0]>test[i,j][1]) and (test[i,j][0]>test[i,j][2]):
#           test[i,j] = np.array([255,255,255,255],dtype="uint8")
#       elif (np.std([test[i,j][0],test[i,j][1],test[i,j][2]])) >= 5 and (test[i,j][1]>test[i,j][0]) and (test[i,j][1]>test[i,j][2]):
#         test[i,j] = np.array([255,255,255,255],dtype="uint8")
#       if np.array_equal(test[i,j],base[i,j]):
#         test[i,j]=np.array([255,255,255,255],dtype="uint8")

#   diff=cv2.cvtColor(test,cv2.COLOR_BGR2GRAY)
#   output_filename = os.path.join(PROCESSED_TRAIN_DATA,filename)
#   output_test=Image.fromarray(diff)
#   output_test.save(output_filename)

In [ ]:
#Double check processed image
for i in range(30000,50002):
  if not os.path.exists(os.path.join(PROCESSED_TRAIN_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")
for i in range(0,25002):
  if not os.path.exists(os.path.join(PROCESSED_TRAIN_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")

In [0]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [0]:
def resnet_v2(input_shape, depth):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tensorflow.keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(1,kernel_initializer='he_normal')(y)
    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [0]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [0]:
# Separate training images into 3 parts: [0-33],(33-66],(66-100]
df1=df[(df.avg_pixel<=cutoff33)]
df2=df[(df.avg_pixel>cutoff33)&(df.avg_pixel<=cutoff66)]
df3=df[(df.avg_pixel>cutoff66)]


In [0]:
print(df1.shape[0],df2.shape[0],df3.shape[0])
# 15325 14582 15093
#df1: 12260:3065
#df2: 11666:2916
#df3: 12075:3018

In [0]:
# Model parameters (prefer not to change)
BATCH_SIZE = 32  # orig paper trained all networks with batch_size=128
EPOCHS = 200
COLORS = 1
DEPTH = COLORS * 9 + 2
input_shape=[256,256,COLORS]

How is the validation split computed?  

If you set the validation_split argument in model.fit to e.g. 0.1, then the validation data used will be the last 10% of the data. If you set it to 0.25, it will be the last 25% of the data, etc. Note that the data isn't shuffled before extracting the validation split, so the validation is literally just the last x% of samples in the input you passed.

In [0]:
# Ignore this block, just use model=load_model('/content/drive/My Drive/Resnet_0408latest_2q.hdf5')
# The following code didn't solve the issue of missing optimizer status

###########
# # Run this if you want to load model
# # Initialize model (remember to re-define generators after this step)
# lr_scheduler_load = LearningRateScheduler(lr_schedule)

# lr_reducer_load = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                                cooldown=0,
#                                patience=5,
#                                min_lr=0.5e-6)
# datagen=ImageDataGenerator(rescale=1./255,validation_split=0.5)
# callbacks_load=[lr_scheduler_load,lr_reducer_load]
# train_gen=datagen.flow_from_dataframe(
#     dataframe=df3[:64],
#     directory='/content/drive/My Drive/proctrain',
#     x_col="id",
#     y_col="clip_count",
#     target_size=(256,256),
#     color_mode="grayscale",
#     class_mode="raw",
#     batch_size=BATCH_SIZE,
#     shuffle=True,
#     subset="training",
#     seed=42
# )
# val_gen=datagen.flow_from_dataframe(
#     dataframe=df3[:64],
#     directory='/content/drive/My Drive/proctrain',
#     x_col="id",
#     y_col="clip_count",
#     target_size=(256,256),
#     color_mode="grayscale",
#     class_mode="raw",
#     batch_size=BATCH_SIZE,
#     shuffle=True,
#     subset="validation",
#     seed=42)

# model=resnet_v2(input_shape=input_shape,depth=DEPTH)
# model.compile(loss='mean_squared_error',
#               optimizer=Adam(learning_rate=lr_schedule(0)),
#               metrics=['accuracy'])

# model.fit(x=train_gen,
#           epochs=2,
#           initial_epoch=0, # Change based on training model
#           callbacks=callbacks_load,
#           validation_data=val_gen,
#           steps_per_epoch=1,
#           validation_steps=1,
#           workers=1,
#           use_multiprocessing=False)
# model.load_weights('/content/drive/My Drive/Resnet_0408latest_2q.hdf5')

In [0]:
#1st quantile

train_datagen=ImageDataGenerator(rescale=1./255,
                horizontal_flip=True,
                vertical_flip=True)

train_gen=train_datagen.flow_from_dataframe(
    dataframe=df1[:12260],
    directory=PROCESSED_TRAIN_DATA,
    x_col="id",
    y_col="clip_count",
    target_size=(256,256),
    color_mode="grayscale",
    class_mode="raw",
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

val_datagen=ImageDataGenerator(rescale=1./255)

val_gen=val_datagen.flow_from_dataframe(
    dataframe=df1[12260:],
    directory=PROCESSED_TRAIN_DATA,
    x_col="id",
    y_col="clip_count",
    target_size=(256,256),
    color_mode="grayscale",
    class_mode="raw",
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42)
STEPS=train_gen.n/train_gen.batch_size
VAL_STEPS=val_gen.n/val_gen.batch_size

model=resnet_v2(input_shape=input_shape,depth=DEPTH)
model.compile(loss='mean_squared_error',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
model_checkpoint=ModelCheckpoint(f'/content/drive/My Drive/Resnet_0407best-1q.hdf5',verbose=1,monitor='val_loss',save_best_only=True,mode="min")# For best model
model_checkpoint2=ModelCheckpoint(f'/content/drive/My Drive/Resnet_0407latest-1q.hdf5',verbose=0,monitor='val_loss',save_best_only=False,mode="min")# For resume training

callbacks = [lr_reducer, lr_scheduler, model_checkpoint, model_checkpoint2]

model.fit(x=train_gen,
          epochs=EPOCHS,
          initial_epoch=0, # Change based on new model or resume training
          callbacks=callbacks,
          validation_data=val_gen,
          steps_per_epoch=STEPS,
          validation_steps=VAL_STEPS,
          workers=1,
          use_multiprocessing=False)

In [0]:
# 2nd quantile
train_datagen=ImageDataGenerator(rescale=1./255,
                horizontal_flip=True,
                vertical_flip=True)

train_gen=train_datagen.flow_from_dataframe(
    dataframe=df2[:11666],
    directory=PROCESSED_TRAIN_DATA,
    x_col="id",
    y_col="clip_count",
    target_size=(256,256),
    color_mode="grayscale",
    class_mode="raw",
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=46 # DO NOT KEEP THE SAME SEED IF RESUME TRAINING
)

val_datagen=ImageDataGenerator(rescale=1./255)

val_gen=val_datagen.flow_from_dataframe(
    dataframe=df2[11666:],
    directory=PROCESSED_TRAIN_DATA,
    x_col="id",
    y_col="clip_count",
    target_size=(256,256),
    color_mode="grayscale",
    class_mode="raw",
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=46 # DO NOT KEEP THE SAME SEED IF RESUME TRAINING
)
STEPS=train_gen.n/train_gen.batch_size
VAL_STEPS=val_gen.n/val_gen.batch_size

# model=resnet_v2(input_shape=input_shape,depth=DEPTH)
# model.compile(loss='mean_squared_error',
#               optimizer=Adam(learning_rate=lr_schedule(0)),
#               metrics=['accuracy'])

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
model_checkpoint=ModelCheckpoint(f'/content/drive/My Drive/Resnet_0408best_2q.hdf5',verbose=1,monitor='val_loss',save_best_only=True,mode="min")# For best model
model_checkpoint2=ModelCheckpoint(f'/content/drive/My Drive/Resnet_0408latest_2q.hdf5',verbose=0,monitor='val_loss',save_best_only=False,mode="min")# For resume training

callbacks = [lr_reducer, lr_scheduler, model_checkpoint, model_checkpoint2]

model.fit(x=train_gen,
          epochs=EPOCHS,
          initial_epoch=148, # Change based on new model or resume training
          callbacks=callbacks,
          validation_data=val_gen,
          steps_per_epoch=STEPS,
          validation_steps=VAL_STEPS,
          workers=1,
          use_multiprocessing=False)

In [0]:
# 3rd quantile
train_datagen=ImageDataGenerator(rescale=1./255,
                horizontal_flip=True,
                vertical_flip=True)

train_gen=train_datagen.flow_from_dataframe(
    dataframe=df3[:12075],
    directory=PROCESSED_TRAIN_DATA,
    x_col="id",
    y_col="clip_count",
    target_size=(256,256),
    color_mode="grayscale",
    class_mode="raw",
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

val_datagen=ImageDataGenerator(rescale=1./255)

val_gen=val_datagen.flow_from_dataframe(
    dataframe=df3[12075:],
    directory=PROCESSED_TRAIN_DATA,
    x_col="id",
    y_col="clip_count",
    target_size=(256,256),
    color_mode="grayscale",
    class_mode="raw",
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42)
STEPS=train_gen.n/train_gen.batch_size
VAL_STEPS=val_gen.n/val_gen.batch_size

model=resnet_v2(input_shape=input_shape,depth=DEPTH)
model.compile(loss='mean_squared_error',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
model_checkpoint=ModelCheckpoint(f'/content/drive/My Drive/Resnet_0410best_3q.hdf5',verbose=1,monitor='val_loss',save_best_only=True,mode="min")# For best model
model_checkpoint2=ModelCheckpoint(f'/content/drive/My Drive/Resnet_0410latest_3q.hdf5',verbose=0,monitor='val_loss',save_best_only=False,mode="min")# For resume training

callbacks = [lr_reducer, lr_scheduler, model_checkpoint, model_checkpoint2]

model.fit(x=train_gen,
          epochs=EPOCHS,
          initial_epoch=0, # Change based on new model or resume training
          callbacks=callbacks,
          validation_data=val_gen,
          steps_per_epoch=STEPS,
          validation_steps=VAL_STEPS,
          workers=1,
          use_multiprocessing=False)

In [0]:
# Prediction creation step start here
#Double check processed image, shold only report 30001 not found
for i in range(25001,30002):
  if not os.path.exists(os.path.join(PROCESSED_TEST_DATA,f"clips-{i}.png")):
    print(f"Error: clips {i} not found")

In [0]:
# Test dataframe modify (load)
df_test=pd.read_csv(os.path.join(WORKING_DIR,'test2.csv'))
#df_test["id"]=df_test['id'].apply(lambda x: f"clips-{x}.png")
df_test

In [0]:
print(cutoff33,cutoff66)
# 7.846227722167969 14.933803405761719
df_test_1q=df_test[df_test.avg_pixel<=cutoff33].reset_index(drop=True)
df_test_2q=df_test[(df_test.avg_pixel>cutoff33)&(df_test.avg_pixel<=cutoff66)].reset_index(drop=True)
df_test_3q=df_test[(df_test.avg_pixel>cutoff66)].reset_index(drop=True)
print(df_test_1q.shape[0],df_test_2q.shape[0],df_test_3q.shape[0])

In [0]:
#Quantile Prediction
def quant_pred(model_path,df_quantile):
  model_quant=load_model(model_path)
  print(f"Loaded model from: {model_path}")
  test_datagen=ImageDataGenerator(rescale=1./255)
  test_gen=test_datagen.flow_from_dataframe(
      dataframe=df_quantile,
      directory='/content/drive/My Drive/proctest',
      x_col="id",
      target_size=(256,256),
      color_mode="grayscale",
      class_mode=None,
      batch_size=1,
      shuffle=False)
  test_steps=test_gen.n/test_gen.batch_size
  test_gen.reset()
  print(f"Predicting {df_quantile.shape[0]} number of obs:")
  pred=model_quant.predict(test_gen,verbose=1,steps=test_steps)
  pred.flatten()
  pred=pd.DataFrame(pred)
  print(pred.shape)
  return pred

In [0]:
pred_1q=quant_pred('/content/drive/My Drive/Resnet_0407best-1q.hdf5',df_test_1q)
pred_2q=quant_pred('/content/drive/My Drive/Resnet_0408best_2q.hdf5',df_test_2q)
pred_3q=quant_pred('/content/drive/My Drive/Resnet_0410best_3q.hdf5',df_test_3q)

In [0]:
df_test_1q=pd.concat([df_test_1q,pred_1q],axis=1,ignore_index=True)
df_test_2q=pd.concat([df_test_2q,pred_2q],axis=1,ignore_index=True)
df_test_3q=pd.concat([df_test_3q,pred_3q],axis=1,ignore_index=True)

submit_df=pd.concat([df_test_1q,df_test_2q,df_test_3q],axis=0,ignore_index=True)
submit_df=submit_df.rename(columns={0:"id",1:"avg_pixel",2:"clip_count"})
submit_df["id"]=submit_df["id"].str.strip("clips-.png")
submit_df=submit_df.sort_values(by="id").reset_index(drop=True).drop(columns="avg_pixel")
submit_df

In [0]:
#Hard-code check if two pictures are identical, if identical, set clip count to 0
base=cv2.imread(os.path.join(TRAIN_DATA,"clips-30047.png"))
changed=0
for index,row in tqdm(submit_df.iterrows()):
  image_id=int(row[0])
  test=cv2.imread(os.path.join(TEST_DATA,f"clips-{image_id}.png"))
  if (np.array_equal(base,test)==True):
    submit_df.loc[index,"clip_count"]=0
    changed+=1
print(changed)

In [0]:
submit_df[submit_df["clip_count"]==0]

In [0]:
submit_df.to_csv('/content/drive/My Drive/Resnet0410_threesplit.csv',index=False)